In [92]:
import os
import numpy as np
import pandas as pd

%matplotlib inline
#stop_words = ["!","\"", "%", "'", "(", ")", "*", "+", "-","，", "：","＜", "＞", "\\", "［", "］", "～", " ﾟ", "￣"]

In [2]:
stop_words_path = 'stopwords/baidu_stopwords.txt'
stop_words_dict = []

with open(stop_words_path, errors='ignore') as f:
    for line in f.readlines():
        stop_words_dict.append(line.strip())
print(len(stop_words_dict))

1396


In [36]:
import re

hh = re.compile(r'[h]+$')
digital = re.compile(r'(iphone)|(ipad)|(honor)|(huawei)|(mini)', re.I)
len(re.findall(iphone, 'iPhone10'))

1

## jieba分词

In [148]:
import re
import jieba

pttn = re.compile(r'[\s\d\<\>\-\*《》#@￥%$_——!！]+')

def cut_word(word):
    words_list = jieba.lcut(word)
    return ' '.join([re.sub(pttn, "", w) for w in words_list if w not in stop_words_dict and len(w) > 0])

def cut_danmu_type(path):
    video_danmu = []
    video_name = []
    for file in os.listdir(path):
        try:
            f = open(os.path.join(path, file),encoding='utf-8')
            df = pd.read_csv(f, header=0)
        except Exception as e:
            print(f"文件读取失败：{file}")
        try:
            df['cut_word'] = df['danmu'].apply(cut_word)
            video_danmu.append(df)
            video_name.append(file.replace('_哔哩哔哩 (゜-゜)つロ 干杯~-bilibili.csv', ''))
        except Exception as e:
            print(f"分词失败{file}")
        f.close()
    return video_danmu, video_name

digital, dname = cut_danmu_type('digital/')
food, fname = cut_danmu_type('food/')
view, vname = cut_danmu_type('view/')
# makeup, mname = cut_danmu_type('makeup/')
cinephile, cname = cut_danmu_type('cinephile/')
news, nname = cut_danmu_type('news/')
# danmu_all, all_name = cut_danmu_type('danmu_all/')
digital


分词失败农  鸣   芒   果_哔哩哔哩 (゜-゜)つロ 干杯~-bilibili.csv
分词失败【半佛】如何把投资人当韭菜割_哔哩哔哩 (゜-゜)つロ 干杯~-bilibili.csv


[                                           danmu  \
 0                                           物理挂啊   
 1                                          还要靠售价   
 2                                        这个感觉太爽了   
 3                                       安卓平板用户暗喜   
 4                           同样的价格是买一个折叠屏还是买两个手机？   
 5                                            用心了   
 6                                       爱死你每个视频了   
 7                                   这个挖槽太冒犯了 - -   
 8                                       好拙劣的对比。。   
 9                                           有意思了   
 10                                         这个真的强   
 11                                         这个太爽了   
 12                                         这个太爽了   
 13                               这个直接发送，不是输入再点发送   
 14                       miui10.1的都可以这么玩，刚刚试了试超爽   
 15                                      有钱真好，嘿嘿嘿   
 16                                          好方便啊   
 17                        三国杀和ygo没有适配，展开后会出问题

## 使用sklearn进行向量化

In [149]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(ddf):
    words_list = list(ddf['cut_word'])
    count_vect = CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b", min_df=1)  #  词至少在1个文档中出现过
    danmu_vec = count_vect.fit_transform(words_list)
    #tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b", min_df=1)  #  词至少在1个文档中出现过
    #danmu_vec = tfidf_vect.fit_transform(words_list)
    return danmu_vec, count_vect.vocabulary_

'''
d1, voc = vectorize(digital[0])
vec_np = np.sum(d1.toarray(),axis=0)
vec_pd = pd.Series(vec_np)
print(len(voc))
voc.items()
zip(voc.values(), voc.keys())
vdict = dict(zip(voc.values(), voc.keys()))
vec_pd = vec_pd.rename(index=vdict)
vec_pd
'''

'\nd1, voc = vectorize(digital[0])\nvec_np = np.sum(d1.toarray(),axis=0)\nvec_pd = pd.Series(vec_np)\nprint(len(voc))\nvoc.items()\nzip(voc.values(), voc.keys())\nvdict = dict(zip(voc.values(), voc.keys()))\nvec_pd = vec_pd.rename(index=vdict)\nvec_pd\n'

In [234]:
def bow(danmu, namelist):
    d_matrix = pd.DataFrame()
    for i in range(len(danmu)):
        try:
            d1, voc = vectorize(danmu[i])
            v_np = np.sum(d1.toarray(),axis=0) # ndarray词向量
            v_pd = pd.Series(v_np)           # pandas词向量
            # 反转字典
            voc.items()
            zip(voc.values(), voc.keys())
            vdict = dict(zip(voc.values(), voc.keys()))
            # 以分出的词作为词向量的索引
            v_pd = v_pd.rename(index=vdict)
            series = v_pd.to_frame().T
            d_matrix = pd.concat([d_matrix, series], axis=0, join='outer', sort=True,
                                ignore_index=False, keys=None, levels=None, names=None)
        except Exception as e:
            namelist.pop(i)
    d_matrix = d_matrix.fillna(0)
    total = d_matrix.sum(axis=1)
    d_matrix = d_matrix.div(total, axis='rows')
        # d_matrix.index = namelist
    return d_matrix, namelist

In [235]:
dmtx, dname = bow(digital, dname)
nmtx, nname = bow(news, nname)
fmtx, fname = bow(food, fname)
vmtx, vname = bow(view, vname)
cmtx, cname = bow(cinephile, cname)

In [240]:
dmtx.index = dname
# nmtx.index = nname
fmtx.index = fname
vmtx.index = vname
cmtx.index = cname

In [156]:
dmtx.to_csv('result/digital.csv', encoding='utf_8_sig')
nmtx.to_csv('result/news.csv', encoding='utf_8_sig')
fmtx.to_csv('result/food.csv', encoding='utf_8_sig')
vmtx.to_csv('result/view.csv', encoding='utf_8_sig')
cmtx.to_csv('result/cinephile.csv', encoding='utf_8_sig')

In [238]:
nmtx.columns

Index(['a', 'ab', 'abc', 'abcde', 'about', 'acab', 'ace', 'action', 'adc',
       'ae',
       ...
       '龙王庙', '龙珠', '龟龟', '발', '씨', 'ａ', 'ｄ', 'ｇ', 'ｐ', 'ﾉ'],
      dtype='object', length=19123)

In [241]:
hh = re.compile(r'^[h]+$')
haha = re.compile(r'^哈[哈]+$')
digital_pttn = re.compile(r'(iphone)|(ipad)|(honor)|(huawei)|(mini)|^(gpu)$|(nvidia)|(ios)', re.I)

In [242]:
def get_df_columns(df):
    return list(df.columns)

def check(df):
    columns = get_df_columns(df)
    # print(columns)
    drop = []
    pttn = re.compile(r'[\u4e00-\u9fa5]+')
    for c in columns:
        target = re.findall(pttn, c)
        if len(target)==0:
            if len(re.findall(digital_pttn, c))==0:
                drop.append(c)
    # print(drop)
    return df.drop(drop, axis=1)

digital_bow = check(dmtx)
food_bow = check(fmtx)
news_bow = check(nmtx)
view_bow = check(vmtx)
cinephile_bow = check(cmtx)

In [179]:
digital_bow.to_csv('result/digital.csv', encoding='utf_8_sig')
news_bow.to_csv('result/news.csv', encoding='utf_8_sig')
food_bow.to_csv('result/food.csv', encoding='utf_8_sig')
view_bow.to_csv('result/view.csv', encoding='utf_8_sig')
cinephile_bow.to_csv('result/cinephile.csv', encoding='utf_8_sig')

In [243]:
cinephile_bow

,t恤,一,一一,一万年,一万次,一上,一下,一下子,一下手,一不小心,...,龙虾,龙里,龙队,龙院,龙须,龙飞凤舞,龙马,龚如心,龟,龟孙
[DW]两位博士见面后竟开始。。。比较音速起子,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
【刘哔】超长版解说！温情解说《汉尼拔》第一季,0.000000,0.000868,0.000000,0.00000,0.00000,0.000000,0.000605,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
"【史蒂芬周】面对可怖的低语者，神奇女孩做出伟大牺牲,一场绝美的时间之战！《神秘博士》解说+解析SE07-13。",0.000000,0.000275,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
【培根悖论】二十年后重温这部影史前三的系列电影开篇作，依旧回味无穷《哈利波特与魔法石》,0.000000,0.000725,0.000000,0.00000,0.00000,0.000000,0.000459,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000062,0.000000,0.000000,0.00000,0.000000,0.000000
【培根悖论】垂直结构监狱600多人疯抢一桌饭，《饥饿站台》见证人性的无限坠落,0.000000,0.000587,0.000000,0.00000,0.00000,0.000000,0.000184,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
【培根悖论】女巫开药带队，蛇王连刀5人！这才是打开《哈利波特与密室》的正确姿势！,0.000000,0.000488,0.000000,0.00000,0.00000,0.000000,0.000205,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
【木鱼微剧场】《三傻大闹宝莱坞》阿米尔汗主演,0.000000,0.000491,0.000000,0.00000,0.00000,0.000000,0.000879,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
【木鱼微剧场】《上帝之城》暴力的循环,0.000000,0.000462,0.000000,0.00000,0.00017,0.000000,0.000598,0.000099,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
【木鱼微剧场】《低俗小说》,0.000000,0.000395,0.000000,0.00000,0.00000,0.000378,0.000414,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
【木鱼微剧场】《傲慢与偏见》,0.000000,0.000463,0.000178,0.00000,0.00000,0.000000,0.000501,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000


## 设定规则匹配重复同义字符串
如 *哈哈* 与 *哈哈哈* 等……

In [248]:
def find_pttn(bow, pttn, col_name):
    col = get_df_columns(bow)
    drop = []
    count = 0
    series = pd.Series()
    for i in range(len(col)):
        if len(re.findall(pttn, col[i]))>0:
            # print(col[i], "\t", end="")
            drop.append(col[i])
            if count == 0:
                series = bow[col[i]]
            else:
                series = series + bow[col[i]]
            count += 1
    bow = bow.drop(drop, axis=1)
    bow[col_name] = series
    return bow

In [249]:
food_bow = find_pttn(food_bow, hh, 'hhh')
digital_bow = find_pttn(digital_bow, hh, 'hhh')
view_bow = find_pttn(view_bow, hh, 'hhh')
cinephile_bow = find_pttn(cinephile_bow, hh, 'hhh')
news_bow = find_pttn(news_bow, hh, 'hhh')

In [251]:
food_bow = find_pttn(food_bow, haha, '哈哈哈')
digital_bow= find_pttn(digital_bow, haha, '哈哈哈')
view_bow = find_pttn(view_bow, haha, '哈哈哈')
cinephile_bow = find_pttn(cinephile_bow, haha, '哈哈哈')
news_bow = find_pttn(news_bow, haha, '哈哈哈')

In [252]:
video_type = {'数码':0, '美食':1, '时政':2, '影评':3, '观点':4}

digital_bow['label'] = 0
food_bow['label'] = 1
cinephile_bow['label'] = 3
view_bow['label'] = 4
news_bow['label'] = 2

In [253]:
# 拼接全部数据

In [301]:
train = pd.concat([digital_bow[:50], food_bow[:50],news_bow[:50]],
                  axis=0, join='outer', sort=True, ignore_index=False, keys=None, levels=None, names=None)
train = train.fillna(0)
print(train.shape)

test = pd.concat([digital_bow[50:60], food_bow[50:60], news_bow[50:60]],
                  axis=0, join='outer', sort=True, ignore_index=False, keys=None, levels=None, names=None)
test = test.fillna(0)
print(test.shape)

(150, 41359)
(30, 41359)


In [302]:
train_X = train.drop(['label'], axis=1)
train_Y = train['label']

feature_names = list(train_X.columns)
target_names = list(video_type.keys())

In [303]:
test_X = test.drop(['label'], axis=1)
test_Y = test['label']

In [259]:
from sklearn import svm
import matplotlib.pyplot as plt

In [260]:
linear = svm.SVC(kernel='linear')
linear = linear.fit(train_X,train_Y)
linear.score(test_X, test_Y)

0.23333333333333334

In [261]:
rbf=svm.SVC()
rbf = ovr.fit(train_X, train_Y)
rbf.score(test_X, test_Y)

0.8333333333333334

In [265]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

train_samples = 150
# Turn up tolerance for faster convergence
clf = LogisticRegression(
    C=50. / train_samples, penalty='l1', solver='saga', tol=0.1
)
clf.fit(train_X, train_Y)
sparsity = np.mean(clf.coef_ == 0) * 100
clf.score(test_X, test_Y)

0.3333333333333333

In [279]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb = gnb.fit(train_X, train_Y)
gnb.score(test_X, test_Y)

0.8333333333333334

In [304]:
indices = np.argsort(np.sum(train_X.values, axis=0))[::-1].tolist()[30:2030]
train_X_1 = train_X.iloc[:, indices]
test_X_1 = test_X.iloc[:, indices]

In [276]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

train_samples = 150
# Turn up tolerance for faster convergence
clf = LogisticRegression(
    C=50. / train_samples, penalty='l1', solver='saga', tol=0.1
)
clf.fit(train_X_1, train_Y)
sparsity = np.mean(clf.coef_ == 0) * 100
clf.score(test_X_1, test_Y)

0.3333333333333333

In [308]:
from sklearn.naive_bayes import GaussianNB

gnb1 = GaussianNB()
gnb1 = gnb.fit(train_X_1, train_Y)
r_gnb1 = gnb.predict(test_X_1)
gnb1.score(test_X_1, test_Y)

0.9666666666666667

In [274]:
linear = svm.SVC(kernel='linear')
linear = linear.fit(train_X_1,train_Y)
linear.score(test_X_1, test_Y)

0.36666666666666664

In [275]:
rbf=svm.SVC()
rbf = ovr.fit(train_X_1, train_Y)
rbf.score(test_X_1, test_Y)

0.9333333333333333

In [233]:
train_X.iloc[:, indices]

,什么,人,真的,喜欢,还,太,没有,好看,牛,不是,...,怎么,hhh,要,厉害,他,少,加油,给,手机,弹幕
1+1 能否大于 2 —— Mate Xs 长期体验,1.186934,1.756517,5.249153,0.000000,4.777660,5.338841,2.287495,3.136069,9.258200,3.788283,...,1.047123,1.230908,2.095291,4.898994,1.466123,0.000000,0.000000,0.374780,7.704930,1.749513
5G终究玩不过百度云,0.000000,0.000000,2.015141,0.000000,1.190318,0.743030,0.389149,0.000000,0.000000,0.307308,...,0.000000,2.317492,0.000000,0.000000,0.334272,0.000000,0.000000,0.000000,0.000000,0.000000
iPhone 11体验：和iPhone 11 Pro的差距只有两个字！,1.428410,1.037533,2.782771,0.000000,2.847545,0.643545,1.793308,1.754753,0.388621,1.273172,...,0.972013,0.000000,0.329678,0.000000,0.295237,0.000000,0.000000,0.000000,0.727229,0.000000
iPhone老用户使用4个月华为mate30 pro的简单感受分享,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P40 VS P40pro VS Mate30pro怎么选？,4.458243,5.546619,14.199550,7.634397,14.793217,3.583276,8.173399,17.467566,3.011703,4.348075,...,1.482972,1.515824,4.809381,1.393710,2.511886,1.367407,1.000000,2.230974,6.134136,2.151446
p40pro拍照真差,0.485918,0.000000,0.000000,0.000000,0.680572,0.928214,0.000000,0.000000,2.261467,0.000000,...,0.226537,0.000000,0.000000,0.000000,0.319516,0.000000,0.000000,0.357383,0.000000,0.000000
P40发布之后，Mate30 Pro变的更香了？年度真香旗舰,1.305171,0.969670,2.528494,3.596448,2.683406,1.011372,3.398433,3.809601,0.000000,1.852074,...,2.141196,0.000000,0.651729,0.698753,0.208220,0.000000,2.035140,4.529804,2.770280,0.000000
「小白测评」 手握一套房！ 华为MateX体验测评 对比三星Fold,10.246056,6.099072,9.236623,6.888434,9.051787,5.640920,7.791298,18.435536,20.013922,10.338324,...,7.891922,1.021896,4.681546,2.104600,2.568943,1.020678,0.000000,3.957030,19.762914,7.089027
「小白测评」vivo X50 Pro体验 把云台装手机里是什么体验？,8.063090,2.405777,6.521575,2.711669,7.407345,1.791588,4.763216,19.047557,13.059904,3.677965,...,4.119124,0.000000,3.307990,3.196901,1.622575,0.000000,0.000000,1.642969,6.616331,1.446534
「小白测评」两万块的开箱 三星Galaxy Z Flip Thom Browne版 听~是RMB的味道~,5.070633,6.557139,10.654722,4.296707,8.956947,9.105831,6.333217,26.885072,1.036872,7.367230,...,7.161004,0.436455,3.130319,0.785137,2.718618,0.563841,1.000000,5.632908,11.535890,4.613255


In [322]:
test_X.shape

(30, 41358)